In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import tensorflow as tf
import random 
from datetime import datetime
import pandas as pd
import os 

import sys
sys.path.append('/glade/work/kjmayer/research/catalyst/ENSOvsMJO/utils/')
# sys.path.append('/glade/u/home/wchapman/ENSOvsMJO/utils/')
from exp_hp import get_hp
from trainGordon_utils import subset, build_model, fullmodel, scheduler, plot_results, adjust_spines
from dataprep_utils import get_testing
sys.path.append('/glade/work/kjmayer/research/catalyst/ENSOvsMJO/interpret/')
# sys.path.append('/glade/u/home/wchapman/ENSOvsMJO/interpret/')
from Gordon_interp import getoutputvecs, confvacc, iconfcorr


# import importlib
# importlib.reload(sys.modules["Gordon_interp"])
# from Gordon_interp import getoutputvecs, confvacc, iconfcorr
# importlib.reload(sys.modules["trainGordon_utils"])
# from trainGordon_utils import subset, build_model, fullmodel, scheduler, plot_results

import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rc('text',usetex=True)
plt.rcParams['font.family']='sans-serif'
plt.rcParams['font.sans-serif']=['Verdana']
plt.rcParams.update({'font.size': 15})
def adjust_spines(ax, spines):
    for loc, spine in ax.spines.items():
        if loc in spines:
            spine.set_position(('outward', 5))
        else:
            spine.set_color('none')
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        ax.yaxis.set_ticks([])
    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
            ax.xaxis.set_ticks([])
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.dpi'] = 150
dpiFig = 300.

2023-12-14 09:29:39.356371: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 09:29:40.746338: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
MODEL_DIR = '/glade/work/kjmayer/research/catalyst/ENSOvsMJO/data/doy_models/' #models/'
# MODEL_DIR = '/glade/scratch/wchapman/ENSOmjo_ML_models/saved_models/'
EXP_NAME = 'default'
hps = get_hp(EXP_NAME)
# variables:
DROPOUT_RATE = hps['DROPOUT_RATE']

MODELNAME1 = 'ENSO'
RIDGE1 = hps['RIDGE1']
HIDDENS1 = hps['HIDDENS1']

MODELNAME2 = 'MJO'
RIDGE2 = hps['RIDGE2']
HIDDENS2 = hps['HIDDENS2']

BATCH_SIZE = hps['BATCH_SIZE']
N_EPOCHS = 10000
PATIENCE = hps['PATIENCE'] # number of epochs of no "improvement" before training is stopped
LR = hps['LR'] # learning rate


LEADS = np.arange(5,6)#,31)
AVGS = np.arange(2,3)#,29) #32)
SEEDS = np.arange(1,2)#,6)

SAVE = True

In [13]:
# Check DOY distribution of confident predictions

for l in LEADS:
    print('LEAD: '+str(l)) 
    for a in AVGS:
        # CHANGE:
        # check if files already exist:
        # ddir_save = '/glade/derecho/scratch/kjmayer/DATA/ENSOvsMJO/data/doy_interpret/'
        # finame_confvsacc = 'confvsacc_DOY_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
        # if not os.path.isfile(ddir_save+finame_confvsacc):
        # print('load testing data')
        X1test, X2test, Ytest = get_testing(N_z500runmean=a,
                                            LEAD=l)

        INPUT_SHAPE1 = np.shape(X1test)[1:][0]
        INPUT_SHAPE2 = np.shape(X2test)[1:][0]


        for s in SEEDS:
            # ENSO MODEL
            model1, input1 = build_model(s,
                                         DROPOUT_RATE,
                                         RIDGE1,
                                         HIDDENS1,
                                         INPUT_SHAPE1,
                                         MODELNAME1)
            # MJO MODEL
            model2, input2 = build_model(s,
                                         DROPOUT_RATE,
                                         RIDGE2,
                                         HIDDENS2,
                                         INPUT_SHAPE2,
                                         MODELNAME2)   
            # COMBINE ENSO & MJO MODEL
            model = fullmodel(model1, model2,
                              input1, input2,
                              s)

            MODEL_FINAME = 'DOY_LEAD_'+str(l)+'_AVG_'+str(a)+'__0000'+str(s)+'.h5'
            model.load_weights(MODEL_DIR+MODEL_FINAME)

            model_rawpreds[s-1] = model.predict((X1test,X2test))                

            conf    = np.max(model_rawpreds[s-1],axis=-1)
            predval = np.argmax(model_rawpreds[s-1],axis=-1)

            # ------- confident predictions --------------------------------------------------------
            per = 80
            conf_thresh = np.percentile(conf,q=per)
            # -------- confident [i_conf_predval] --------
            i_conf_predval = np.where(conf > conf_thresh)[0]
            
            X1test_doy =  X1test[:,-1]
            X2test_doy =  X2test[:,-1]
            X1test_conf_doy = X1test[i_conf_predval,-1]
            X2test_conf_doy = X2test[i_conf_predval,-1]
            
            # ----- model contribution: ------------------------------------------------------------
            model1_rawpreds, model2_rawpreds = getoutputvecs(model,
                                                                   model1,
                                                                   model2,
                                                                   X1test,
                                                                   X2test)
            # model X winning class (model X output * weight)
            model1pred = np.argmax(model1_rawpreds,axis=1)
            model2pred = np.argmax(model2_rawpreds,axis=1)
            
            
            # Where ENSO/MJO/final model (model 1/model 2/total) are correct
            i_model1_confcorr = model1pred[i_conf_predval]==Ytest[i_conf_predval]
            i_model2_confcorr = model2pred[i_conf_predval]==Ytest[i_conf_predval]
            # i_model_confcorr  = predval[i_conf_predval]==Ytest[i_conf_predval]
                        
            X1test_confcorr_doy = X1test_conf_doy[i_model1_confcorr]
            X2test_confcorr_doy = X2test_conf_doy[i_model2_confcorr]
                
                
# CHANGE:                
#             if SAVE:
#                 # print('saving')
#                 ddir_save = '/glade/derecho/scratch/kjmayer/DATA/ENSOvsMJO/data/doy_interpret/'
                
#                 finame_confvsacc = 'confvsacc_DOY_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_confvsacc, confvsacc)


LEAD: 5
361/361 [==============================] - 1s 1ms/step


In [11]:
i_model1_confcorr.shape

(2307,)

In [12]:
X1test_conf_doy.shape

(2307,)

In [14]:
np.arange(1,366)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [ ]:
vals_doy_test = np.array(X2_testxr_mem_NDJF['time.dayofyear'])
vals_doy_test[vals_doy_test<60]+=366
vals_doy_test = (vals_doy_test-np.mean(vals_doy_test))/np.std(vals_doy_test)